# Time series forcasting using ARIMA

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
#from statsmodels.tsa.arima_model import ARIMA
import statsmodels.tsa.arima.model as sm
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams.update({'figure.figsize':(10,3), 'figure.dpi':120})

In [ ]:
# Import data (usage of website)
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/wwwusage.csv', names=['value'], header=0)

In [ ]:
df.head()

## Find d (Order of differencing)

In [ ]:
# Original Series
fig, axes = plt.subplots(1, 2)
axes[0].plot(df["value"])
axes[0].set_title('Original Series')
plot_acf(df["value"], ax=axes[1])
plt.show()

In [ ]:
# 1st Differencing
fig, axes = plt.subplots(1, 2)
axes[0].plot(df["value"].diff())
axes[0].set_title('Original Series')
plot_acf(df["value"].diff().dropna(), ax=axes[1])
plt.show()

In [ ]:
# 2nd Differencing
fig, axes = plt.subplots(1, 2)
axes[0].plot(df["value"].diff().diff())
axes[0].set_title('Original Series')
plot_acf(df["value"].diff().diff().dropna(), ax=axes[1])
plt.show()

For the above series, the time series reaches stationarity with two orders of differencing. But on looking at the autocorrelation plot for the 2nd differencing the lag goes into the far negative zone fairly quick, which indicates, the series might have been over differenced. So, I am going to tentatively fix the order of differencing as 1 even though the series is not perfectly stationary (weak stationarity).

## Find p (order in AR terms)

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].plot(df["value"].diff())
axes[0].set_title('1st Differencing')
plot_pacf(df["value"].diff().dropna(), ax=axes[1])
plt.show()

You can observe that the PACF lag 1 is quite significant since is well above the significance line. Lag 2 turns out to be significant as well, slightly managing to cross the significance limit (blue region). But I am going to be conservative and tentatively fix the p as 1.

## Find q (order of MA)

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].plot(df["value"].diff())
axes[0].set_title('1st Differencing')
plot_acf(df["value"].diff().dropna(), ax=axes[1])
plt.show()

Couple of lags are well above the significance line. So, let’s tentatively fix q as 2. When in doubt, go with the simpler model that sufficiently explains the Y.

## 1,1,2 ARIMA Model
- p = 1 (AR order)
- d = 1 (Difference)
- q = 2 (MA order) 

In [ ]:
model = sm.ARIMA(df["value"], order=(1,1,2))
model_fit = model.fit()
print(model_fit.summary())

The model summary reveals a lot of information. The table in the middle is the coefficients table where the values under ‘coef’ are the weights of the respective terms. 

The Akaike Information Critera (AIC) is a widely used measure of a statistical model. It basically quantifies 1) the goodness of fit, and 2) the simplicity/parsimony, of the model into a single statistic. When comparing two models, the one with the lower AIC is generally “better”.


Notice here the coefficient of the MA2 term is close to zero and the P-Value in ‘P>|z|’ column is highly insignificant. It should ideally be less than 0.05 for the respective X to be significant. So, let’s rebuild the model without the MA2 term.

## 1,1,1 ARIMA Model
- p = 1 (AR order)
- d = 1 (Difference)
- q = 1 (MA order) 


In [ ]:
# 1,1,1 ARIMA Model
model = sm.ARIMA(df.value, order=(1,1,1))
model_fit = model.fit()
print(model_fit.summary())

The model AIC has reduced, which is good. The P Values of the AR1 and MA1 terms have improved and are highly significant (<< 0.05).

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid[1:])
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

The residual errors seem fine with near zero mean and uniform variance.

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(df[1:],label="Original")
ax.plot(model_fit.predict(1,len(df)),label="Fitted")
ax.legend()

In [ ]:
# Prediction
model_fit.predict(0,101)

### Forecasting

In [ ]:
#Forecast
num_forecast = 10
#Y forecast
num_forecast_array = model_fit.predict(len(df),len(df)+num_forecast)
#X forecast
x_forecast = np.arange(len(df),len(df)+num_forecast+1)

fig, ax = plt.subplots(1,1)
ax.plot(df[1:],label="Original")
ax.plot(model_fit.predict(1,len(df)),label="Fitted")
ax.plot(np.arange(len(df),len(df)+num_forecast+1), num_forecast_array,label="Fitted",marker='o', linewidth=2, markersize=3)
ax.legend()

## Auto ARIMA

In [ ]:
import pmdarima as pm

In [ ]:
model = pm.auto_arima(df.value, start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

In [ ]:
model.plot_diagnostics(figsize=(7,5))
plt.show()

In [ ]:
n_periods = 10
fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = np.arange(len(df.value), len(df.value)+n_periods)

# make series for plotting purpose
fc_series = pd.Series(fc, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

# Plot
plt.plot(df.value)
plt.plot(fc_series, color='darkgreen',marker='o', linewidth=2, markersize=3)
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("Final Forecast of WWW Usage")
plt.show()